In [1]:
import torch
from torchvision.transforms.functional import to_tensor,center_crop
from PIL.Image import open as imopen
from skimage.metrics import peak_signal_noise_ratio as psnr
from os.path import join
from os import listdir
from models.dpirUnet import DPIRNNclass,PotentialNNclass,REDPotentialNNclass,GSPNPNNclass
import numpy as np

In [2]:
testTensor=torch.stack([center_crop(to_tensor(imopen(join('miscs','CBSD10',n)).convert('RGB')),(256,256)) for n in listdir(join('miscs','CBSD10'))],dim=0)

In [3]:
gspnp=GSPNPNNclass(numInChan=3,numOutChan=3,network='unet',train_network=False)

In [4]:
gspnp.network.load_state_dict(torch.load('miscs/unet.pt'))

<All keys matched successfully>

In [5]:
red=REDPotentialNNclass(numInChan=3,numOutChan=3,network='unet',train_network=False)

In [6]:
red.network.load_state_dict(torch.load('../potential_exp/red/denoiser/epoch_749.pt',map_location='cpu'))

<All keys matched successfully>

In [7]:
sigmaLst=[2.55,7.65,12.75]

In [8]:
print('gspnp and red comparison on CBSD10')
gspnp.network.eval()
red.network.eval()
for sigma in sigmaLst:
    noise=torch.randn_like(testTensor)*sigma/255.
    sigmaTensor=torch.tensor(sigma,dtype=testTensor.dtype,device=testTensor.device)/255.
    NoisytestTensor=testTensor+noise
    gspnpRecon=gspnp(NoisytestTensor,sigmaTensor,create_graph=False,strict=False)
    redRecon=red(NoisytestTensor,sigmaTensor,create_graph=False,strict=False)
    gspnpPSNR=np.stack([psnr(gspnpRecon[i].detach().numpy().transpose(1,2,0),testTensor[i].detach().numpy().transpose(1,2,0),data_range=1.0) for i in range(testTensor.shape[0])],axis=0).mean()
    redPSNR=np.stack([psnr(redRecon[i].detach().numpy().transpose(1,2,0),testTensor[i].detach().numpy().transpose(1,2,0),data_range=1.0) for i in range(testTensor.shape[0])],axis=0).mean()
    print('sigma:',sigma,'gspnp:',gspnpPSNR,'red:',redPSNR)

gspnp and red comparison on CBSD10
sigma: 2.55 gspnp: 44.567157659845186 red: 43.80000518492583
sigma: 7.65 gspnp: 37.90528089301971 red: 37.63699155720627
sigma: 12.75 gspnp: 35.04465616790055 red: 34.81687193919423
